<a href="https://colab.research.google.com/github/ThiagSampaio/DeepLearning/blob/main/Rede_Neural_Profunda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Construindo uma rede neural profunda: Passo a passo

Vamos construir uma rede neural profunda com inúmeras camadas.

**Notação**:
- Sobrescrito $[l]$ denota uma quantidade associada ao $l^{th}$ camada. 
    - Exemplo: $ a ^ {[L]} $ é a ativação da camada $ L ^ {th} $. $ W ^ {[L]} $ e $ b ^ {[L]} $ são os $ L ^ {th} $ parâmetros da camada.
- Sobrescrito $ (i) $ denota uma quantidade associada ao exemplo $ i ^ {th} $.
    - Exemplo: $ x ^ {(i)} $ é o $ i ^ {th} $ exemplo de treinamento.
- Lowerscript $ i $ denota a entrada $ i ^ {th} $ de um vetor.
    - Exemplo: $ a ^ {[l]} _ i $ denota a $ i ^ {th} $ entrada das ativações da camada $ l ^ {th} $).



<a name='1'> </a>
## 1 - Pacotes

Vamos importar todos os pacotes que usaremos neste projeto.

- [numpy] (www.numpy.org) é o pacote principal para computação científica com Python.
- [matplotlib] (http://matplotlib.org) é uma biblioteca para plotar gráficos em Python.
- dnn_utils fornece algumas funções necessárias para este notebook.
- testCases fornece alguns casos de teste para avaliar a exatidão de suas funções
- np.random.seed (1) é usado para manter consistentes todas as chamadas de função aleatórias.

In [5]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases import *
from dnn_utils import sigmoid, sigmoid_backward, relu, relu_backward
from public_tests import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

<a name='2'> </a>
## 2 - Esboço

Para construir sua rede neural, implementaremos várias "funções auxiliares". Essas funções auxiliares serão usadas na próxima tarefa para construir uma rede neural de duas camadas e uma rede neural de camada L.

Cada pequena função auxiliar terá instruções detalhadas das etapas necessárias. Aqui está um esboço das etapas desta tarefa:

- Inicializaremos os parâmetros para uma rede de duas camadas e para uma rede neural de $ L $ -camada
- Implementaremos o módulo de propagação direta (mostrado em roxo na figura abaixo)
     - Completaremos a parte LINEAR da etapa de propagação direta de uma camada (resultando em $ Z ^ {[l]} $).
     - A função ATIVAÇÃO é fornecida para nós (relu / sigmóide)
     - Combinaremos as duas etapas anteriores em uma nova função de avanço [LINEAR-> ATIVAÇÃO].
     - Empilharemos a função de avanço [LINEAR-> RELU] vez L-1 (para as camadas 1 a L-1) e adicionaremos um [LINEAR-> SIGMOID] no final (para a camada final $ L $). Isso nos dará uma nova função L_model_forward.
- Calcularemos a perda
- Implementaremos o módulo de propagação para trás (denotado em vermelho na figura abaixo)
    - Completaremos a parte LINEAR da etapa de propagação para trás de uma camada
    - O gradiente da função ACTIVATE é fornecido para nós (relu_backward / sigmoid_backward)
    - Combinenaremos as duas etapas anteriores em uma nova função de retrocesso [LINEAR-> ATIVAÇÃO]
    - Empilharemos [LINEAR-> RELU] para trás L-1 vezes e adicionaremos [LINEAR-> SIGMOID] para trás em uma nova função L_model_backward
- Por fim, atualizaremos os parâmetros

![](https://drive.google.com/uc?export=view&id=1ZJbpt-D3NHnXu9i5_0cnCGjO9WiZk8zK)

**Observação**:

Para cada função de avanço, há uma função de retrocesso correspondente. É por isso que em cada etapa de seu módulo de encaminhamento armazenaremos alguns valores em um cache. Esses valores em cache são úteis para calcular gradientes.

No módulo de retropropagação, podemos usar o cache para calcular os gradientes.

<a name='3'></a>
## 3 - Inicialização

Escreveremos duas funções auxiliares para inicializar os parâmetros do seu modelo. A primeira função será usada para inicializar parâmetros para um modelo de duas camadas. O segundo generaliza esse processo de inicialização para camadas $ L $.

<a name='3-1'></a>
### 3.1 - Rede Neural de 2 camadas 

<a name='ex-1'></a>
### Demonstração 1 - initialize_parameters

Criaremos e inicializaremos os parâmetros da rede neural de 2 camadas.

** Instruções **:

- A estrutura do modelo é: * LINEAR -> RELU -> LINEAR -> SIGMOID *.
- Use esta inicialização aleatória para as matrizes de peso: `np.random.randn (shape) * 0,01` com a forma correta
- Use inicialização zero para os vieses: `np.zeros (shape)`

In [13]:
def initialize_parameters (n_x, n_h, n_y):
  """
    Argumento:
    n_x - tamanho da camada de entrada
    n_h - tamanho da camada oculta
    n_y - tamanho da camada de saída
    
    Retorna:
    parameters -- dicionário python contendo seus parâmetros:
                    W1 - matriz de peso da forma (n_h, n_x)
                    b1 - vetor de polarização da forma (n_h, 1)
                    W2 - matriz de peso da forma (n_y, n_h)
                    b2 - vetor de polarização da forma (n_y, 1)
  """
    
  np.random.seed(1)

  W1 = np.random.randn(n_h, n_x) * 0.01
  b1 = np.zeros((n_h, 1))
  W2 = np.random.randn(n_y, n_h) * 0.01
  b2 = np.zeros((n_y, 1))

  assert (W1.shape == (n_h, n_x))
  assert (b1.shape == (n_h, 1))
  assert (W2.shape == (n_y, n_h))
  assert (b2.shape == (n_y, 1))
    
  parameters = {"W1": W1,
                "b1": b1,
                "W2": W2,
                "b2": b2}
    
  return parameters

In [14]:
parameters = initialize_parameters(3,2,1)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

initialize_parameters_test(initialize_parameters)

W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.01744812 -0.00761207]]
b2 = [[0.]]
 All tests passed.


<a name='3-2'> </a>
### 3.2 - Rede Neural de camada L

A inicialização para uma rede neural de camada L mais profunda é mais complicada porque há muito mais matrizes de peso e vetores de polarização. Ao completar a função `initialize_parameters_deep`, você deve se certificar de que suas dimensões correspondem entre cada camada. Lembre-se de que $ n ^ {[l]} $ é o número de unidades na camada $ l $. Por exemplo, se o tamanho de sua entrada $ X $ for $ (12288, 209) $ (com $ m = 209 $ exemplos), então:

<table style="width:100%">
    <tr>
        <td>  </td> 
        <td> <b>Shape of W</b> </td> 
        <td> <b>Shape of b</b>  </td> 
        <td> <b>Activation</b> </td>
        <td> <b>Shape of Activation</b> </td> 
    <tr>
    <tr>
        <td> <b>Layer 1</b> </td> 
        <td> $(n^{[1]},12288)$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td> 
        <td> $(n^{[1]},209)$ </td> 
    <tr>
    <tr>
        <td> <b>Layer 2</b> </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr>
       <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>  
   <tr>
       <td> <b>Layer L-1</b> </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
   <tr>
   <tr>
       <td> <b>Layer L</b> </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>
</table>

Lembre-se de que quando calculamos $ W X + b $ em python, ele realiza a transmissão. Por exemplo, se:

$$ W = \begin{bmatrix}
    w_{00}  & w_{01} & w_{02} \\
    w_{10}  & w_{11} & w_{12} \\
    w_{20}  & w_{21} & w_{22} 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    x_{00}  & x_{01} & x_{02} \\
    x_{10}  & x_{11} & x_{12} \\
    x_{20}  & x_{21} & x_{22} 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    b_0  \\
    b_1  \\
    b_2
\end{bmatrix}\tag{2}$$

Depois $WX + b$ será:

$$ WX + b = \begin{bmatrix}
    (w_{00}x_{00} + w_{01}x_{10} + w_{02}x_{20}) + b_0 & (w_{00}x_{01} + w_{01}x_{11} + w_{02}x_{21}) + b_0 & \cdots \\
    (w_{10}x_{00} + w_{11}x_{10} + w_{12}x_{20}) + b_1 & (w_{10}x_{01} + w_{11}x_{11} + w_{12}x_{21}) + b_1 & \cdots \\
    (w_{20}x_{00} + w_{21}x_{10} + w_{22}x_{20}) + b_2 &  (w_{20}x_{01} + w_{21}x_{11} + w_{22}x_{21}) + b_2 & \cdots
\end{bmatrix}\tag{3}  $$

<a name='ex-2'> </a>
### Demonstração 2 - initialize_parameters_deep

Implementação da inicialização para uma rede neural de camada L.


In [15]:
def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- array python (lista) contendo as dimensões de cada camada em nossa rede
    
    Returns:
    parameters --   dicionário python contendo seus parâmetros "W1", "b1", ..., "WL", "bL":
                    Wl - matriz de peso da forma (layer_dims [l], layer_dims [l-1])
                    bl - vetor de polarização da forma (layer_dims [l], 1)
    """
    
    np.random.seed(3)
    parameters = {}

    # número de camadas em uma rede
    L = len(layer_dims) 

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
        
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [16]:
parameters = initialize_parameters_deep([5,4,3])

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

initialize_parameters_deep_test(initialize_parameters_deep)

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]
 All tests passed.


<a name='4'> </a>
## 4 - Módulo de propagação direta

<a name='4-1'> </a>
### 4.1 - Linear para frente

Agora que você inicializou seus parâmetros, pode fazer o módulo de propagação direta. Comece implementando algumas funções básicas que você pode usar novamente mais tarde, ao implementar o modelo. Agora, você completará três funções nesta ordem:

- LINEAR
- LINEAR -> ATIVAÇÃO onde ATIVAÇÃO será ReLU ou Sigmóide.
- [LINEAR -> RELU] $ \ times $ (L-1) -> LINEAR -> SIGMÓIDE (modelo completo)

O módulo linear progressivo (vetorizado sobre todos os exemplos) calcula as seguintes equações:

$$ Z ^ {[l]} = W ^ {[l]} A ^ {[l-1]} + b ^ {[l]}\tag {4}$$

onde $ A ^ {[0]} = X $.

<a name='ex-3'> </a>
### Demonstração 3 - linear_forward

Construiremos a parte linear da propagação direta.

**Lembrete**:
A representação matemática desta unidade é $ Z ^ {[l]} = W ^ {[l]} A ^ {[l-1]} + b ^ {[l]} $.

In [17]:
def linear_forward(A, W, b):
    """
    Implemente a parte linear da propagação direta de uma camada.

    Argumentos:
    A - ativações da camada anterior (ou dados de entrada): (tamanho da camada anterior, número de exemplos)
    W - matriz de pesos: matriz numpy de forma (tamanho da camada atual, tamanho da camada anterior)
    b - vetor de polarização, matriz numpy de forma (tamanho da camada atual, 1)

    Retorna:
    Z - a entrada da função de ativação, também chamada de parâmetro de pré-ativação
    cache - uma tupla python contendo "A", "W" e "b"; armazenado para calcular o passe para trás de forma eficiente
    """
    
    Z = np.dot(W,A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [18]:
t_A, t_W, t_b = linear_forward_test_case()
t_Z, t_linear_cache = linear_forward(t_A, t_W, t_b)
print("Z = " + str(t_Z))

linear_forward_test(linear_forward)

Z = [[ 3.26295337 -1.23429987]]
 All tests passed.


<a name='4-2'> </a>
### 4.2 - Linear-Ativação para Frente

Neste notebook, usaremos duas funções de ativação:

- ** Sigmóide **: $ \ sigma (Z) = \ sigma (W A + b) = \frac{1}{1 + e ^ {- (W A + b)}} $. 
Foi fornecida a função `sigmoid` que retorna dois itens: o valor de ativação" `a`" e um "` cache` "que contém" `Z`" (é o que iremos alimentar a função retrógrada correspondente). Para usá-lo, basta chamar:
`` `python
A, cache_de_ativação = sigmóide (Z)
`` `

- ** ReLU **: A fórmula matemática para ReLu é $ A = RELU (Z) = max (0, Z) $`. Esta função retorna ** dois ** itens: o valor de ativação "` A` "e um" `cache`" que contém "` Z` "(é o que você irá alimentar para a função de retrocesso correspondente). Para usá-lo, basta chamar:

```python
A, cache_de_ativação = relu (Z)
```

Para maior comodidade, iremos agrupar duas funções (Linear e Ativação) em uma função (LINEAR-> ATIVAÇÃO). Portanto, implementaremos uma função que faz o passo de avanço LINEAR, seguido por um passo de avanço de ATIVAÇÃO.

<a name='ex-4'></a>
### Demonstração 4 - linear_activation_forward

Implementaremos a propagação direta da camada * LINEAR-> ATIVAÇÃO *. A relação matemática é: $ A ^ {[l]} = g (Z ^ {[l]}) = g (W ^ {[l]} A ^ {[l-1]} + b ^ {[l]} ) $ onde a ativação "g" pode ser sigmóide () ou relu (). Usaremos `linear_forward ()` e a função de ativação correta.

In [19]:
def linear_activation_forward (A_prev, W, b, activation):
    """
    Implementar a propagação direta para a camada LINEAR-> ATIVAÇÃO

    Argumentos:
    A_prev - ativações da camada anterior (ou dados de entrada): (tamanho da camada anterior, número de exemplos)
    W - matriz de pesos: matriz numpy de forma (tamanho da camada atual, tamanho da camada anterior)
    b - vetor de polarização, matriz numpy de forma (tamanho da camada atual, 1)
    activation - a ativação a ser usada nesta camada, armazenada como uma string de texto: "sigmóide" ou "relu"

    Retorna:
    A - a saída da função de ativação, também chamada de valor pós-ativação
    cache - uma tupla python contendo "linear_cache" e "activation_cache";
             armazenado para calcular o passe para trás de forma eficiente
    """
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward (A_prev, W, b)
        A, activation_cache = sigmoid (Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward (A_prev, W, b)
        A, activation_cache = relu (Z)

    assert (A.shape == (W.shape [0], A_prev.shape [1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [20]:
t_A_prev, t_W, t_b = linear_activation_forward_test_case()

t_A, t_linear_activation_cache = linear_activation_forward(t_A_prev, t_W, t_b, activation = "sigmoid")
print("With sigmoid: A = " + str(t_A))

t_A, t_linear_activation_cache = linear_activation_forward(t_A_prev, t_W, t_b, activation = "relu")
print("With ReLU: A = " + str(t_A))

linear_activation_forward_test(linear_activation_forward)

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]
 All tests passed.
